Before you turn this lab in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
DRIVER = "Aubrey"
NAVIGATOR = "Jonathan"

# Optical Character Recognition Lab

Welcome to the optical character recognition lab! By the end of this lab you will have

- *Trained a neural network to perform optical character recognition*

Let's get started!

In [2]:
def passed(): print('✅')

# Load Data

The following code loads and normalizes the MNIST dataset and filters it down to a set containing only exemplar images.

In [3]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, RepeatVector, Flatten, Conv2D, Conv1D, TimeDistributed

Using TensorFlow backend.


In [4]:
from IPython.display import display
from keras.preprocessing.image import array_to_img
from keras.datasets import mnist
import numpy as np
np.random.seed(1337)

[X_mnist, y_mnist], _ = mnist.load_data()
X_mnist = X_mnist/255.
digit_idxs = [np.argwhere(y_mnist == d).flatten()[0] for d in range(10)]
X_mnist = X_mnist[digit_idxs]

for x in X_mnist:
    img = array_to_img(np.expand_dims(x, axis=-1))
    display(img)

# Generate Training Data

The following code generates sequences of the exemplar MNIST digits along with their labels and provides the option to change two hyperparameters.

In [5]:
NB_TRAIN = 90
T = 2

In [6]:
import itertools
from keras.preprocessing.image import array_to_img

H = W = 28
X, y = np.zeros([NB_TRAIN, H, W*T]), [None]*NB_TRAIN
i = 0
D = set()
while i < NB_TRAIN:
    digits = np.random.choice(10, size=T, replace=False)
    if tuple(digits) in D:
        continue
    D.add(tuple(digits))
    X[i], y[i] = np.hstack([x for x in X_mnist[digits]]), digits
    i += 1 
X = np.expand_dims(X, axis=-1)

Y = np.zeros([NB_TRAIN, T, 10])
for i, ys in enumerate(y):
    for j, y_ in enumerate(ys):
        Y[i, j, y_] = 1

for idx in range(5):
    x = X[idx]
    img_ = array_to_img(X[idx])
    print(y[idx])
    display(img_)
    
X_train, Y_train = X[NB_TRAIN//10:], Y[NB_TRAIN//10:]
X_val, Y_val = X[:NB_TRAIN//10], Y[:NB_TRAIN//10]

print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

[6 3]


[7 5]


[7 4]


[4 2]


[9 0]


(81, 28, 56, 1) (81, 2, 10)
(9, 28, 56, 1) (9, 2, 10)


In [7]:
Y_train[0]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

# Task

- Define and compile a deep learning model to perform optical character recognition on the MNIST sequence dataset

# Requirements

- Save your model into a python variable called `model`

In [8]:
model = Sequential()
model.add(Flatten(input_shape=X[0].shape))
model.add(RepeatVector(2, ))
model.add(SimpleRNN(128,return_sequences=True))

#model.add(TimeDistributed(Conv2D(filters=100,kernel_size=2, )))
model.add(TimeDistributed(Dense(10,activation='softmax')))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 2, 1568)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 2, 128)            217216    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2, 10)             1290      
Total params: 218,506.0
Trainable params: 218,506.0
Non-trainable params: 0.0
_________________________________________________________________


In [9]:
model.predict(X_val[:2])

array([[[ 0.04645525,  0.05979344,  0.0792594 ,  0.1813675 ,  0.05678769,
          0.16752505,  0.14332481,  0.03393608,  0.1569138 ,  0.07463706],
        [ 0.03223978,  0.04787417,  0.03038925,  0.14522775,  0.0354295 ,
          0.18770312,  0.2739906 ,  0.03409065,  0.11934557,  0.09370958]],

       [[ 0.03015554,  0.04650818,  0.05530696,  0.18264122,  0.06076873,
          0.13091095,  0.2059682 ,  0.06528342,  0.16613498,  0.05632189],
        [ 0.01999741,  0.05352768,  0.0231866 ,  0.12057216,  0.06733397,
          0.19185671,  0.29124615,  0.0477159 ,  0.12636954,  0.05819394]]], dtype=float32)

In [10]:
model.output_shape

(None, 2, 10)

In [11]:
 w = model.layers[2].get_weights()

In [12]:
28*28

784

In [13]:
w[0].shape

(1568, 128)

In [14]:
w[1].shape

(128, 128)

In [15]:
w[2].shape

(128,)

## a

# Model Tests

In [16]:
(None,) + X[0].shape

(None, 28, 56, 1)

In [17]:
assert model.input_shape == (None,) + X[0].shape
assert model.output_shape == (None,) + Y[0].shape
assert model.count_params() > 100_000
assert model.loss == 'categorical_crossentropy'
assert model.metrics == ['accuracy']
assert model.predict(X[:1])[0].shape == Y[0].shape

passed()

✅


# Task

- Optimize `model` on `X_train` and `Y_train`

In [18]:
model.fit(X_train,Y_train,validation_data=(X_val,Y_val), epochs=20)

Train on 81 samples, validate on 9 samples
Epoch 1/20
81/81 [==============================] - 0s - loss: 2.3260 - acc: 0.2346 - val_loss: 1.7431 - val_acc: 0.5000
Epoch 2/20
81/81 [==============================] - 0s - loss: 1.4921 - acc: 0.6481 - val_loss: 1.2002 - val_acc: 0.7222
Epoch 3/20
81/81 [==============================] - 0s - loss: 0.9520 - acc: 0.8580 - val_loss: 0.7980 - val_acc: 0.8333
Epoch 4/20
81/81 [==============================] - 0s - loss: 0.6175 - acc: 0.9630 - val_loss: 0.5606 - val_acc: 0.9444
Epoch 5/20
81/81 [==============================] - 0s - loss: 0.4194 - acc: 0.9938 - val_loss: 0.4143 - val_acc: 1.0000
Epoch 6/20
81/81 [==============================] - 0s - loss: 0.2921 - acc: 1.0000 - val_loss: 0.3046 - val_acc: 1.0000

# Performance Tests

In [19]:
loss, acc = model.evaluate(X_val, Y_val)
assert acc > 0.10
passed()

9/9 [==============================] - 0s
✅


In [20]:
loss, acc = model.evaluate(X_val, Y_val)
assert acc > 0.15
passed()

9/9 [==============================] - 0s
✅


In [21]:
loss, acc = model.evaluate(X_val, Y_val)
assert acc > 0.20
passed()

9/9 [==============================] - 0s
✅


In [22]:
loss, acc = model.evaluate(X_val, Y_val)
assert acc > 0.25
passed()

9/9 [==============================] - 0s
✅


In [23]:
loss, acc = model.evaluate(X_val, Y_val)
assert acc > 0.30
passed()

9/9 [==============================] - 0s
✅


In [24]:
loss, acc = model.evaluate(X_val, Y_val)
assert acc > 0.50
passed()

9/9 [==============================] - 0s
✅


In [25]:
loss, acc = model.evaluate(X_val, Y_val)
assert acc > 0.75
passed()

9/9 [==============================] - 0s
✅


In [26]:
loss, acc = model.evaluate(X_val, Y_val)
assert acc > 0.90
passed()

9/9 [==============================] - 0s
✅
